<center><h1><font size=6> Initial Data Processing and Feature Engineering </h1></center>

This notebook takes the raw EPL match data that was scraped in Notebook 01 and processes the data to make sure it is in a clean and consistent format, including:
* Cleaning up column names and descriptions
* Cleaning column types
* Ensuring team names are consistent across the dataset

It then takes the processed data and calculates some simple features that I thought were useful based on my domain knowledge of the EPL, including:

* **Days since last game and number of games in last 21 days**: this is an important indicator of fatigue.
* **Points, total points and goal difference in the current season**. This is a good indicator of how well a team is doing in the current season.
* **League position**: this is an important indicator of form. 
* **Stats from previous season**: this is a good indicator of how good a team is.
* **Last head to head results**: the most recent result and form of the team against the same opponent (and at the same venue)
* **Promoted dummy variable**: this could be an important indicator and is a useful variable to capture as promoted teams are likely to have less available data on historic form so will need to be treated slightly differently.

More machine learning specific processing and feature engineering is done after the Exploratory Data Analysis (EDA) phase, but I wanted to do this after I split the data into training and testing sets to prevent any snooping bias.

### Load libraries and setup notebook configuration

In [1]:
# import packages
import pandas as pd 
import numpy as np
import os
from pathlib import Path
import warnings
import datetime


# set pandas configurations
pd.set_option("display.precision", 2) # display to 1 decimpal place
pd.set_option("display.max.columns", None) # display all columns so we can view the whole dataset
pd.set_option('display.float_format', '{:.2f}'.format) # Disable scientific notation for pandas
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning) # Disable setting with copy warnings


# set directories
os.chdir('..') # change current working directory to the parent directory to help access files/directories at a higher level
DATAPATH = Path(r'data') # set data path


# import from source directory
from src import constants

### Load data from local data file

In [2]:
# load EPL match data
matches_raw = pd.read_csv(f"{DATAPATH}/raw/matches_long_raw.csv")

matches_raw.head(5)

C:\Users\lucel\AppData\Local\Temp\ipykernel_15028\3869096252.py:2: DtypeWarning: Columns (2,15,16,24) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_raw = pd.read_csv(f"{DATAPATH}/raw/matches_long_raw.csv")


,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,season,team,date_downloaded,Rk,Squad,MP,W,D,L,GD,Pts,Pts/MP,Top Team Scorer,Goalkeeper
0,0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1,3,Liverpool,NaN,NaN,57.00,NaN,Rúben Dias,4-3-3,Craig Pawson,Match Report,NaN,2022,Manchester City,2023-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,2.20,0.50,75.00,62443.00,İlkay Gündoğan,4-3-3,Michael Oliver,Match Report,NaN,2022,Manchester City,2023-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,1.70,0.10,67.00,53453.00,İlkay Gündoğan,4-2-3-1,David Coote,Match Report,NaN,2022,Manchester City,2023-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,2.10,1.80,69.00,52258.00,İlkay Gündoğan,4-3-3,Jarred Gillett,Match Report,NaN,2022,Manchester City,2023-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,2.20,0.10,74.00,53112.00,Kevin De Bruyne,4-2-3-1,Darren England,Match Report,NaN,2022,Manchester City,2023-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<center><h1><font size=6> Data Processing </h1></center>

### Clean data

In [3]:
# collect only relevant columns (some we drop because of inconsistent reporting (e.g., only being reported in very early seasons))
matches_clean = matches_raw[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain',
                             'Formation', 'Referee', 'Match Report', 'Notes', 'season', 'team','date_downloaded']]


# clean column names
matches_clean.columns = map(str.lower, matches_clean.columns) # make columns lower case
matches_clean.columns = matches_clean.columns.str.replace(' ', '_') # replace spaces in column names

# Convert date into datetime object
matches_clean['date'] = pd.to_datetime(matches_clean['date'])

# Convert time into time object 
matches_clean['time'] = pd.to_datetime(matches_clean['time'], format='%H:%M').dt.time

# Convert date downloaded into datetime object 
matches_clean['date_downloaded'] = pd.to_datetime(matches_clean['date_downloaded'])

# Convert goals for to numeric 
matches_clean.loc[:, 'gf'] = pd.to_numeric(matches_clean['gf'], errors='coerce', downcast='integer')

# Convert goals against to numeric 
matches_clean.loc[:, 'ga'] = pd.to_numeric(matches_clean['ga'], errors='coerce', downcast='integer')

# convert matchweek into a single number
matches_clean['round'] = matches_clean['round'].str.extract(r'Matchweek (\d+)', expand=False).fillna(matches_clean['round'])

# Remove any rows without a date, team, or opponent
columns_to_check = ['team', 'opponent', 'date']
matches_clean = matches_clean.dropna(subset=columns_to_check)

### Create a unique game ID for each game

In [4]:
# create an ID based on the date, venue, team and opponent
matches_clean['unique_match_id'] = (matches_clean['date'].dt.strftime("%Y%m%d")
                                    + pd.factorize(matches_clean['venue'])[0].astype(str)
                                    + pd.factorize(matches_clean['team'])[0].astype(str) 
                                    + pd.factorize(matches_clean['opponent'])[0].astype(str))
matches_clean['unique_match_id'] = matches_clean['unique_match_id'].astype('int64')
matches_clean = matches_clean[['unique_match_id'] + [col for col in matches_clean.columns if col != 'unique_match_id']] # make ID first column

# Remove rows with duplicate values in 'unique_match_id' column
matches_clean.drop_duplicates(subset='unique_match_id', inplace=True)

### Cleaning team names

One issue we have with the data in its raw format is that the team names are different in the 'team' and 'opponent' column, which makes it dificult to map across data in future. So let's create a mapping to ensure all our team names are consistent.

In [5]:
# collect team names
pl_matches = matches_clean[matches_clean['comp'] == "Premier League"] # filter to just contain PL opponents otherwise we get european and cup game opponents as well

teams = np.sort(pl_matches.team.unique())
opponents = np.sort(pl_matches.opponent.unique())


# find the index points within each list where the names are not equal
unequal_indexes = [i for i, (x, y) in enumerate(zip(teams, opponents)) if x != y]

for index in unequal_indexes:
    print(f"Team name [{index}]: {teams[index]}, Opponenent name [{index}]: {opponents[index]}")

Team name [4]: Blackburn Rovers, Opponenent name [4]: Blackburn
Team name [6]: Bolton Wanderers, Opponenent name [6]: Bolton
Team name [10]: Brighton and Hove Albion, Opponenent name [10]: Brighton
Team name [13]: Charlton Athletic, Opponenent name [13]: Charlton Ath
Team name [20]: Huddersfield Town, Opponenent name [20]: Huddersfield
Team name [27]: Manchester United, Opponenent name [27]: Manchester Utd
Team name [29]: Newcastle United, Opponenent name [29]: Newcastle Utd
Team name [31]: Nottingham Forest, Opponenent name [31]: Nott'ham Forest
Team name [34]: Queens Park Rangers, Opponenent name [34]: QPR
Team name [36]: Sheffield United, Opponenent name [36]: Sheffield Utd
Team name [37]: Sheffield Wednesday, Opponenent name [37]: Sheffield Weds
Team name [43]: Tottenham Hotspur, Opponenent name [43]: Tottenham
Team name [45]: West Bromwich Albion, Opponenent name [45]: West Brom
Team name [46]: West Ham United, Opponenent name [46]: West Ham
Team name [49]: Wolverhampton Wanderers

In [6]:
# map the opponent team names to the team names to ensure consistency
team_mapping = dict(zip(opponents, teams)) # create a dictionary mapping each unique opponent name to the team name
matches_clean['opponent'] = matches_clean['opponent'].map(lambda x: team_mapping.get(x, x)) # map to new names but leave original name if no mapping found (e.g., champions league games)

<center><h1><font size=6> Basic Feature Engineering </h1></center>

In [7]:
matches = matches_clean.copy().sort_values(['team', 'season', 'date']).reset_index(drop=True)

### Extract day of the week the game is played

In [8]:
matches['day_of_week'] = matches['date'].dt.strftime('%a')

### Days since last game

In [9]:
# calculate days since last game
matches['days_since_last_game'] = matches.groupby(['team', 'season'])['date'].diff().dt.days

### Amount of games played in last 21 days

In [10]:
# calculate count of games in previous 21 days up to current day for each team
matches['games_played_last_21_days'] = matches.set_index('date')\
                .groupby('team', sort=False)['unique_match_id']\
                .rolling('21d', closed='left').count().tolist()

### Points, total points and goal difference

In [11]:
# calculate the number of points in each game based on the result
# Function to calculate points from a game
def calculate_points(row):
    if row['result'] == 'W':
        return 3
    elif row['result'] == 'D':
        return 1
    else:
        return 0

# calculate points
matches['points'] = matches.apply(calculate_points, axis=1)


# define a function to calculate the cumulative amount of a certain variable over the course of a PL season (up to but not including the game of the row)
def calculate_cumulative_pl_value(data, new_column_name, column):
    data[new_column_name] = data[data['comp'] == 'Premier League'].groupby(['team', 'season'])[column].transform(lambda x: x.shift().cumsum())
    
    # Identify the transition points between seasons
    season_transition_mask = data['round'] == '1'
    season_start_mask = season_transition_mask & (data['comp'] == 'Premier League')
    
    # Reset the cumulative values at the start of each new season
    data.loc[season_start_mask, new_column_name] = 0
    
    return data

    
# calculate total cumulative points, goals for and goals against
matches = calculate_cumulative_pl_value(data=matches, new_column_name='pl_total_points', column='points')
matches = calculate_cumulative_pl_value(data=matches, new_column_name='pl_total_gf', column='gf')
matches = calculate_cumulative_pl_value(data=matches, new_column_name='pl_total_ga', column='ga')


# calculate total cumulative goal difference
matches['pl_total_goal_diff'] = matches['pl_total_gf'] - matches['pl_total_ga']

columns_to_convert = ['pl_total_points', 'pl_total_gf', 'pl_total_ga', 'pl_total_goal_diff']
matches[columns_to_convert] = matches[columns_to_convert].astype(float)

### Relative position in the PL table at each matchweek

In [12]:
def calculate_league_position(data):
    # Create a copy of the dataframe to avoid modifying the original one
    temp_df = data[data['comp'] == 'Premier League'].copy()

    # Convert 'round' to integer matchweek number
    temp_df['matchweek'] = temp_df['round']

    # Create a new dataframe to hold the ranks
    ranks = pd.DataFrame()

    # For each season and each matchweek, sort teams by total points, then by goal difference and finally by team name
    for season in temp_df['season'].unique():
        for matchweek in temp_df['round'].unique():
            temp = temp_df[(temp_df['season'] == season) & (temp_df['matchweek'] == matchweek)].copy()
            temp.sort_values(['pl_total_points', 'pl_total_goal_diff', 'team'], ascending=[False, False, True], inplace=True)
            temp['rank'] = range(1, len(temp) + 1)
            temp = temp[temp['rank'] <= 20]  # Discard ranks greater than 20
            ranks = pd.concat([ranks, temp])

    # Replace rank of first matchweek with NaN in the 'temp_df' dataframe
    ranks.loc[ranks['matchweek'] == 1, 'rank'] = np.nan

    # For each matchweek and each team, take the minimum rank
    ranks['rank'] = ranks.groupby(['season', 'matchweek', 'team'])['rank'].transform('min')
    
    # Merge back with the original dataframe
    matches = data.merge(ranks[['unique_match_id', 'rank']], on='unique_match_id', how='left')

    return matches

matches = calculate_league_position(matches)

matches = matches.rename(columns = {'rank': 'pl_position'})
matches['pl_position'] = matches['pl_position'].astype(float)

### Head to Head

In [13]:
# Create a sorted copy of the data
matches_sorted = matches.sort_values(['date']).copy()

# Create a set identifier for each match (sorted alphabetically to ensure that the same set represents the same pair of teams regardless of home/away status)
matches_sorted['set_identifier'] = matches_sorted.apply(lambda row: ''.join(sorted([row['team'], row['opponent']])), axis=1)

# Calculate last h2h result
matches_sorted['last_h2h'] = matches_sorted.groupby('set_identifier')['points'].transform(lambda x: x.shift())

# Calculate average points from last 5 head-to-head games
matches_sorted['last_h2h_form'] = matches_sorted.groupby('set_identifier')['points'].rolling(window=5, min_periods=1, closed = 'left').mean().reset_index(level=0, drop=True)

# Create a venue identifier (including the venue in the identifier)
matches_sorted['venue_identifier'] = matches_sorted.apply(lambda row: ''.join(sorted([row['team'], row['opponent'], row['venue']])), axis=1)

# Calculate last h2h result at same venue
matches_sorted['last_h2h_venue'] = matches_sorted.groupby('venue_identifier')['points'].transform(lambda x: x.shift())

# Calculate average points from last 5 head-to-head games at the same venue
matches_sorted['last_h2h_venue_form'] = matches_sorted.groupby('venue_identifier')['points'].rolling(window=5, min_periods=1, closed = 'left').mean().reset_index(level=0, drop=True)

matches = matches_sorted.sort_values(['date']).copy()
columns_to_convert = ['last_h2h', 'last_h2h_form', 'last_h2h_venue', 'last_h2h_venue_form']
matches[columns_to_convert] = matches[columns_to_convert].astype(float)

### PL stats from previous season

In [14]:
# Sum all the points, goals scored, goals conceded, and calculate goal difference for each team in each season
season_stats = matches[matches['comp'] == 'Premier League'].groupby(['season', 'team']).agg({
    'points': 'sum',
    'gf': 'sum',
    'ga': 'sum'
}).reset_index()

# Rename the variables and increment the season by one
season_stats['season'] = season_stats['season'] + 1
season_stats = season_stats.rename(columns={'points': 'prev_season_points', 'gf': 'prev_season_gf', 'ga': 'prev_season_ga'})

# Merge with the match data
matches = matches.merge(season_stats, on=['season', 'team'], how='left')

# calculate previous season goal difference
matches['prev_season_goal_diff'] = matches['prev_season_gf'] - matches['prev_season_ga']
columns_to_convert = ['prev_season_points', 'prev_season_gf', 'prev_season_ga', 'prev_season_goal_diff']
matches[columns_to_convert] = matches[columns_to_convert].astype(float)

### Form over last 5 games

In [15]:
# Define a list of variables to calculate the form
variables = ['points', 'gf', 'ga', 'poss', 'xg', 'xga']

# Filter the dataframe to include only Premier League games
premier_league_matches = matches[matches['comp'] == 'Premier League'].copy()

# Calculate the form for each variable
for variable in variables:
    # Create a new column for the variable's form
    premier_league_matches[f'{variable}_pl_form'] = premier_league_matches.groupby('team')[variable].rolling(window=5, min_periods=1, closed='left').mean().reset_index(level=0, drop=True)


# create a list of form variables
form_variables =  [f'{variable}_pl_form' for variable in variables]
    
# Merge the form data into the original matches dataframe
matches = matches.merge(premier_league_matches[['unique_match_id'] + form_variables], on='unique_match_id', how='left')
columns_to_convert = form_variables
#['points_pl_form', 'gf_pl_form', 'ga_pl_form', 'poss_pl_form', 'xg_pl_form', 'xga_pl_form']
matches[columns_to_convert] = matches[columns_to_convert].astype(float)

### Dummy variable for promoted teams

In [16]:
# Create an empty column 'promoted' in the matches dataframe
matches['promoted'] = 0

# Get the unique seasons in the dataframe
unique_seasons = matches['season'].unique()

# Iterate over the unique seasons
for i, season in enumerate(unique_seasons):
    if i > 0: # ignore the first season
        prev_season_teams = matches[matches['season'] == unique_seasons[i - 1]]['team'].unique()
        current_season_teams = matches[matches['season'] == season]['team']
        
        # Update the 'promoted' column for teams not present in the previous season
        matches.loc[(matches['season'] == season) & (~current_season_teams.isin(prev_season_teams)), 'promoted'] = 1

### Collecting opponent data


In [17]:
# Select the columns to calculate for the opponent
opponent_columns = ['date', 'team', 'opponent', 'days_since_last_game', 'games_played_last_21_days', 'pl_total_points', 'pl_total_gf',
                    'pl_total_ga', 'pl_total_goal_diff', 'pl_position', 'points_pl_form', 'gf_pl_form',
                    'ga_pl_form', 'poss_pl_form', 'xg_pl_form', 'xga_pl_form', 'prev_season_points',
                    'prev_season_gf', 'prev_season_ga', 'prev_season_goal_diff', 'promoted']


# Merge original data with data from the opponent
matches = matches.merge(matches[opponent_columns], left_on=['date', 'team', 'opponent'], right_on=['date', 'opponent', 'team'],
                          suffixes=('', '_opponent'))

### Make venue column binary

In [18]:
matches['home'] = np.where(matches['venue'] == 'Home', 1, 0)

### Collect only relevant data

In [19]:
# filter to just contain premier league data
matches = matches[matches['comp'] == 'Premier League'].sort_values(['date'])

# collect only relevant variables for ML, i.e., nothing based on the actual game data as we won't have that information when predicting games
matches_final = matches[['unique_match_id','season','date', 'day_of_week','round','day','team','promoted','opponent','promoted_opponent','home','points',
                        'days_since_last_game','games_played_last_21_days','pl_total_points','pl_total_gf','pl_total_ga','pl_total_goal_diff','pl_position','last_h2h','last_h2h_form','last_h2h_venue','last_h2h_venue_form',
                        'prev_season_points','prev_season_gf','prev_season_ga','prev_season_goal_diff','points_pl_form','gf_pl_form','ga_pl_form','poss_pl_form','xg_pl_form','xga_pl_form',
                        'days_since_last_game_opponent','games_played_last_21_days_opponent','pl_total_points_opponent','pl_total_gf_opponent','pl_total_ga_opponent','pl_total_goal_diff_opponent','pl_position_opponent',
                         'points_pl_form_opponent','gf_pl_form_opponent', 'ga_pl_form_opponent', 'poss_pl_form_opponent','xg_pl_form_opponent', 'xga_pl_form_opponent','prev_season_points_opponent', 'prev_season_gf_opponent',
                         'prev_season_ga_opponent', 'prev_season_goal_diff_opponent']]

# remove the first premier league season from the data frame as we dont have enough data
matches_final = matches_final[matches_final['season'] != 1993].reset_index(drop=True)

# convert round to int
matches_final['round'] = matches_final['round'].astype(int)                                                                 

matches_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22362 entries, 0 to 22361
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   unique_match_id                     22362 non-null  int64         
 1   season                              22362 non-null  int64         
 2   date                                22362 non-null  datetime64[ns]
 3   day_of_week                         22362 non-null  object        
 4   round                               22362 non-null  int32         
 5   day                                 22362 non-null  object        
 6   team                                22362 non-null  object        
 7   promoted                            22362 non-null  int64         
 8   opponent                            22362 non-null  object        
 9   promoted_opponent                   22362 non-null  int64         
 10  home                  

In [20]:
# save clean data in processed data file
matches_final.to_csv(f"{DATAPATH}/processed/matches_processed.csv", index=False)

In [21]:
matches_final.head(5)

,unique_match_id,season,date,day_of_week,round,day,team,promoted,opponent,promoted_opponent,home,points,days_since_last_game,games_played_last_21_days,pl_total_points,pl_total_gf,pl_total_ga,pl_total_goal_diff,pl_position,last_h2h,last_h2h_form,last_h2h_venue,last_h2h_venue_form,prev_season_points,prev_season_gf,prev_season_ga,prev_season_goal_diff,points_pl_form,gf_pl_form,ga_pl_form,poss_pl_form,xg_pl_form,xga_pl_form,days_since_last_game_opponent,games_played_last_21_days_opponent,pl_total_points_opponent,pl_total_gf_opponent,pl_total_ga_opponent,pl_total_goal_diff_opponent,pl_position_opponent,points_pl_form_opponent,gf_pl_form_opponent,ga_pl_form_opponent,poss_pl_form_opponent,xg_pl_form_opponent,xga_pl_form_opponent,prev_season_points_opponent,prev_season_gf_opponent,prev_season_ga_opponent,prev_season_goal_diff_opponent
0,1993081414182,1994,1993-08-14,Sat,1,Sat,Ipswich Town,0,Oldham Athletic,0,0,3,NaN,NaN,0.00,0.00,0.00,0.00,7.00,3.00,1.50,3.00,1.50,52.00,50.00,55.00,-5.00,1.20,1.60,1.80,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,14.00,2.00,2.00,2.00,NaN,NaN,NaN,49.00,63.00,74.00,-11.00
1,199308141441,1994,1993-08-14,Sat,1,Sat,Wimbledon,0,West Ham United,1,0,3,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,54.00,56.00,55.00,1.00,1.00,1.20,1.60,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993081411512,1994,1993-08-14,Sat,1,Sat,Everton,0,Southampton,0,0,3,NaN,NaN,0.00,0.00,0.00,0.00,6.00,1.00,1.25,1.00,0.50,53.00,53.00,55.00,-2.00,1.00,1.60,1.80,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,18.00,0.80,1.20,2.00,NaN,NaN,NaN,50.00,54.00,61.00,-7.00
3,1993081422234,1994,1993-08-14,Sat,1,Sat,Sheffield United,0,Swindon Town,1,1,3,NaN,NaN,0.00,0.00,0.00,0.00,16.00,NaN,NaN,NaN,NaN,52.00,54.00,53.00,1.00,2.00,2.00,1.20,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,19.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993081413516,1994,1993-08-14,Sat,1,Sat,Blackburn Rovers,0,Chelsea,0,0,3,NaN,NaN,0.00,0.00,0.00,0.00,3.00,0.00,1.25,0.00,0.50,71.00,68.00,46.00,22.00,2.40,2.00,1.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,4.00,1.20,1.60,2.20,NaN,NaN,NaN,56.00,51.00,54.00,-3.00
